In [1]:
!wget "https://data.sdss.org/sas/dr17/env/APOGEE_STARHORSE/APOGEE_DR17_EDR3_STARHORSE_v2.fits"

--2025-06-19 11:49:29--  https://data.sdss.org/sas/dr17/env/APOGEE_STARHORSE/APOGEE_DR17_EDR3_STARHORSE_v2.fits
Resolving data.sdss.org (data.sdss.org)... 155.101.19.133
Connecting to data.sdss.org (data.sdss.org)|155.101.19.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 289100160 (276M) [application/octet-stream]
Saving to: ‘APOGEE_DR17_EDR3_STARHORSE_v2.fits’

APOGEE_DR17_EDR3_ST 100%[===================>] 275.71M  5.64MB/s    in 45s     

2025-06-19 11:50:15 (6.12 MB/s) - ‘APOGEE_DR17_EDR3_STARHORSE_v2.fits’ saved [289100160/289100160]



In [27]:
from astropy.table import Table

data = Table.read("APOGEE_DR17_EDR3_STARHORSE_v2.fits")

In [28]:
from mmoma.datasets.astropile import FastAstroPileLoader

dm = FastAstroPileLoader(
    "/mnt/home/polymathic/ceph/MultimodalUniverse/gaia_v2/", 
    dataset_name="dr3_xp", 
    include_healpix=[2068],
    num_workers=8,
)
dm.setup("")
loader = dm.train_dataloader()

In [29]:
import torch
import numpy as np
from tqdm import tqdm

t = Table.read("APOGEE_DR17_EDR3_STARHORSE_v2.fits")

starhorse_ids = np.array(t['EDR3_source_id']).astype(int)
targets = np.array(t[['teff50', 'logg50', 'met50']]).view("<f4").reshape(len(t), 3)

In [32]:
saved_gaia_ids = []
saved_bp = []
saved_rp = []
saved_labels = []

for batch in tqdm(loader):
    gaia_ids = np.asarray(batch['object_id'], dtype=int)
    _, gaia_ix, starhorse_ix = np.intersect1d(gaia_ids, starhorse_ids, return_indices=True)

    saved_gaia_ids.append(gaia_ids[gaia_ix])
    saved_bp.append(batch['bp_coefficients'][gaia_ix])
    saved_rp.append(batch['rp_coefficients'][gaia_ix])
    saved_labels.append(targets[starhorse_ix])

saved_gaia_ids = np.concatenate(saved_gaia_ids)
saved_bp = np.concatenate(saved_bp)
saved_rp = np.concatenate(saved_rp)
saved_labels = np.concatenate(saved_labels)

592it [00:25, 23.09it/s]


In [33]:
new_table = Table()
new_table['gaia_source_id'] = saved_gaia_ids
new_table['bp_coefficients'] = saved_bp
new_table['rp_coefficients'] = saved_rp
new_table['teff50'] = saved_labels[:, 0]
new_table['logg50'] = saved_labels[:, 1]
new_table['met50'] = saved_labels[:, 2]
new_table.write("gaia_prop.fits", overwrite=True)

In [34]:
from mmoma.datasets.astropile import CrossMatchedAstroPileLoader
from mmoma.datasets.preprocessing import PadSpectra
import matplotlib.pyplot as plt

dm = CrossMatchedAstroPileLoader(
    left_dataset_path="/mnt/ceph/users/polymathic/MultimodalUniverse/desi",
    right_dataset_path="/mnt/ceph/users/flanusse/myGaia/gaia.py",
    left_dataset_name=None,
    right_dataset_name="parallax_sample",
    formatting_fns=[
        PadSpectra(max_length=7800),
    ],
    batch_size=16,
    num_workers=8,
    exclude_healpix=None,
    include_healpix=[1708,1709,1643,1640,1642,2698,1081,74,2096,132],
)
dm.setup("")
loader = dm.train_dataloader()

batch = next(iter(loader))

Number of common healpix regions:  7
